In [375]:
%reset -sf

In [376]:
# LOAD THIS BEFORE START, run rmdl

#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# import numpy as np
import scipy
import matplotlib.pyplot as plt

abc = "abcdefghijklmnopqrstuvwxyz"
abc_map = {c:i for i,c in enumerate(abc)}
M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [377]:
with open("/Users/htong/Downloads/input.txt") as f:
    srr = f.read().strip().split("\n\n")

In [378]:
start = ["tl", "gd", "zb", "gc"]

srr = """
%cg -> mt, hb
%sp -> xm
%nr -> hf, mt
&qz -> qn
%df -> hd
%vg -> rm, kx
%gm -> mt, md
%ls -> hc
%lq -> zq, fx
&zd -> bz, kg, zb, lf, sq, zk, jx
%lz -> mt
%sq -> zk
%zn -> kx, tc
&zq -> mb, hc, qz, ql, tl, ls
&mt -> zm, tt, mh, gd, md
%lm -> mb, zq
%hf -> mt, sm
%hb -> mh, mt
%rm -> kx
%gc -> kx, sp
&cq -> qn
%mh -> jt
%zm -> nr
%xm -> kx, ld
&jx -> qn
&qn -> rx
%mp -> qt, kx
%zk -> vj
%hd -> mp, kx
%tl -> zq, hl
%zb -> zd, ph
%cl -> zd
&tt -> qn
%ld -> zn
%js -> lq, zq
%sm -> mt, lz
%qt -> vg, kx
%md -> cg
%vj -> bz, zd
%qs -> zd, fs
%mb -> ps
&kx -> cq, gc, sp, df, ld
%hc -> lm
%tc -> df, kx
%ps -> js, zq
%fs -> qc, zd
%hl -> jj, zq
%bz -> qs
%jj -> zq, ql
%ql -> ls
%ph -> kg, zd
%qc -> cl, zd
%lf -> sq
%kg -> lf
%fx -> zq
%jt -> zm, mt
%gd -> gm, mt
""".strip().split("\n")

In [379]:
# start = ["a", "b", "c"]

# srr = """
# %a -> b
# %b -> c
# %c -> inv
# &inv -> a
# """.strip().split("\n")

In [380]:
# start = ["a"]

# srr = """
# %a -> inv, con
# &inv -> b
# %b -> con
# &con -> output
# """.strip().split("\n")

In [381]:
flipflop = {}
conjuction = {}

for row in srr:
    if row[0] == '%':
        a,b = row.split(" -> ")
        a = a[1:]
        arr = b.split(", ")
        flipflop[a] = arr
    if row[0] == '&':
        a,b = row.split(" -> ")
        a = a[1:]
        arr = b.split(", ")
        conjuction[a] = arr

In [382]:
flipflop

{'cg': ['mt', 'hb'],
 'sp': ['xm'],
 'nr': ['hf', 'mt'],
 'df': ['hd'],
 'vg': ['rm', 'kx'],
 'gm': ['mt', 'md'],
 'ls': ['hc'],
 'lq': ['zq', 'fx'],
 'lz': ['mt'],
 'sq': ['zk'],
 'zn': ['kx', 'tc'],
 'lm': ['mb', 'zq'],
 'hf': ['mt', 'sm'],
 'hb': ['mh', 'mt'],
 'rm': ['kx'],
 'gc': ['kx', 'sp'],
 'mh': ['jt'],
 'zm': ['nr'],
 'xm': ['kx', 'ld'],
 'mp': ['qt', 'kx'],
 'zk': ['vj'],
 'hd': ['mp', 'kx'],
 'tl': ['zq', 'hl'],
 'zb': ['zd', 'ph'],
 'cl': ['zd'],
 'ld': ['zn'],
 'js': ['lq', 'zq'],
 'sm': ['mt', 'lz'],
 'qt': ['vg', 'kx'],
 'md': ['cg'],
 'vj': ['bz', 'zd'],
 'qs': ['zd', 'fs'],
 'mb': ['ps'],
 'hc': ['lm'],
 'tc': ['df', 'kx'],
 'ps': ['js', 'zq'],
 'fs': ['qc', 'zd'],
 'hl': ['jj', 'zq'],
 'bz': ['qs'],
 'jj': ['zq', 'ql'],
 'ql': ['ls'],
 'ph': ['kg', 'zd'],
 'qc': ['cl', 'zd'],
 'lf': ['sq'],
 'kg': ['lf'],
 'fx': ['zq'],
 'jt': ['zm', 'mt'],
 'gd': ['gm', 'mt']}

In [383]:
conjuction

{'qz': ['qn'],
 'zd': ['bz', 'kg', 'zb', 'lf', 'sq', 'zk', 'jx'],
 'zq': ['mb', 'hc', 'qz', 'ql', 'tl', 'ls'],
 'mt': ['zm', 'tt', 'mh', 'gd', 'md'],
 'cq': ['qn'],
 'jx': ['qn'],
 'qn': ['rx'],
 'tt': ['qn'],
 'kx': ['cq', 'gc', 'sp', 'df', 'ld']}

In [384]:
conjuction_input_prev = {x:{} for x in conjuction}

for k,v in flipflop.items():
    for nex in v:
        if nex in conjuction:
            conjuction_input_prev[nex][k] = False

In [385]:
conjuction_input_prev

{'qz': {},
 'zd': {'zb': False,
  'cl': False,
  'vj': False,
  'qs': False,
  'fs': False,
  'ph': False,
  'qc': False},
 'zq': {'lq': False,
  'lm': False,
  'tl': False,
  'js': False,
  'ps': False,
  'hl': False,
  'jj': False,
  'fx': False},
 'mt': {'cg': False,
  'nr': False,
  'gm': False,
  'lz': False,
  'hf': False,
  'hb': False,
  'sm': False,
  'jt': False,
  'gd': False},
 'cq': {},
 'jx': {},
 'qn': {},
 'tt': {},
 'kx': {'vg': False,
  'zn': False,
  'rm': False,
  'gc': False,
  'xm': False,
  'mp': False,
  'hd': False,
  'qt': False,
  'tc': False}}

In [386]:
state_ff = {x:False for x in flipflop}  # is_on

In [387]:
low_count = 0
high_count = 0

cnts = defaultdict(list)

for idx in range(1, 100000):
    queue = deque([(x, False, None) for x in start])  # val, is_high
    low_count += 1

    while queue:
        x, is_high, prev = queue.popleft()
        if is_high:
            high_count += 1
        else:
            low_count += 1
            
#         print(x, is_high)
        if x == "qn" and is_high == True:
            cnts[prev].append(idx)
#             print(idx, prev)

        if x == "rz" and is_high == True:
            print(idx)
        
        if x in flipflop:
            if is_high:
                continue
            if state_ff[x]:  # is on
                state_ff[x] = not state_ff[x]
                for nex in flipflop[x]:
                    queue.append([nex, False, x])
                    if nex in conjuction_input_prev:
                        conjuction_input_prev[nex][x] = False
            else:
                state_ff[x] = not state_ff[x]
                for nex in flipflop[x]:
                    queue.append([nex, True, x])
                    if nex in conjuction_input_prev:
                        conjuction_input_prev[nex][x] = True

        if x in conjuction:
            all_high = True
            for k,v in conjuction_input_prev[x].items():
                if not v:
                    all_high = False
            if all_high:
                for nex in conjuction[x]:
                    queue.append([nex, False, x])
                    if nex in conjuction_input_prev:
                        conjuction_input_prev[nex][x] = False
            else:
                for nex in conjuction[x]:
                    queue.append([nex, True, x])
                    if nex in conjuction_input_prev:
                        conjuction_input_prev[nex][x] = True

In [388]:
3907 * 3911 * 3931 * 4021

241528477694627

In [390]:
cnts

defaultdict(list,
            {'jx': [3907,
              7814,
              11721,
              15628,
              19535,
              23442,
              27349,
              31256,
              35163,
              39070,
              42977,
              46884,
              50791,
              54698,
              58605,
              62512,
              66419,
              70326,
              74233,
              78140,
              82047,
              85954,
              89861,
              93768,
              97675],
             'qz': [3911,
              7822,
              11733,
              15644,
              19555,
              23466,
              27377,
              31288,
              35199,
              39110,
              43021,
              46932,
              50843,
              54754,
              58665,
              62576,
              66487,
              70398,
              74309,
              78220,
              82131,
       